# PyTorch Pretrained Models for IoT Deployment
This notebook explores pretrained models for efficient deployment on IoT devices, focusing on model optimization and quantization. We analyze the relationship between batch size, inference time, and accuracy using six architectures: ResNet18, ResNet50, VGG16, MobileNetV2, DenseNet121, and ViT-B/16.

## 1. Import Required Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import time
import numpy as np
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

## 2. Load a Subset of the ImageNet Dataset
For demonstration, we use a small subset of ImageNet or a similar dataset (e.g., CIFAR-10 or ImageNet mini) due to resource constraints.

In [2]:
# For demonstration, use CIFAR-10 as a proxy for ImageNet subset
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
subset_indices = list(range(200))  # Use 200 images for quick evaluation
test_subset = Subset(testset, subset_indices)

## 3. Select and Load Pretrained Models (Six Architectures)
We use ResNet18, ResNet50, VGG16, MobileNetV2, DenseNet121, and ViT-B/16.

In [3]:
model_names = [
    'resnet18',
    'resnet50',
    'vgg16',
    'mobilenet_v2',
    'densenet121',
    'vit_b_16'
]
models_dict = {}
for name in model_names:
    if hasattr(models, name):
        model = getattr(models, name)(pretrained=True)
        model.eval()
        models_dict[name] = model

/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weight

## 4. Evaluate Models: Batch Size, Inference Time, and Accuracy
We evaluate each model with different batch sizes and record inference time and accuracy.

In [4]:
def evaluate_model(model, dataloader, device):
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Batches", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    end_time = time.time()
    accuracy = 100 * correct / total
    inference_time = end_time - start_time
    return accuracy, inference_time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_sizes = [1, 4, 8, 16, 32]
results = []
for name in tqdm(models_dict.keys(), desc="Models"):  # Progress bar for models
    model = models_dict[name]
    model.to(device)
    for batch_size in tqdm(batch_sizes, desc=f"{name} batch sizes", leave=False):  # Progress bar for batch sizes
        loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)
        acc, inf_time = evaluate_model(model, loader, device)
        results.append({'Model': name, 'Batch Size': batch_size, 'Accuracy': acc, 'Inference Time': inf_time})
df = pd.DataFrame(results)
df

Models:   0%|          | 0/6 [00:00<?, ?it/s]





























































































































































































































































































































































Batches:  80%|████████  | 20/25 [00:13<00:07,  1.49s/it]











Batches:   0%|          | 0/13 [00:00<?, ?it/s]































Batches:   0%|          | 0/7 [00:00<?, ?it/s]













Models:  17%|█▋        | 1/6 [01:18<06:30, 78.08s/it]

Models:  17%|█▋        | 1/6 [01:18<06:30, 78.08s/it]



































































































































































































































































































## 5. Visualize Results
Plot the relationship between batch size, inference time, and accuracy for each model.

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))
for name in model_names:
    subset = df[df['Model'] == name]
    ax1.plot(subset['Batch Size'], subset['Inference Time'], label=f'{name} (Time)')
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Inference Time (s)')
ax1.legend()
plt.title('Inference Time vs Batch Size for Each Model')
plt.show()
# Accuracy plot
fig, ax2 = plt.subplots(figsize=(10, 6))
for name in model_names:
    subset = df[df['Model'] == name]
    ax2.plot(subset['Batch Size'], subset['Accuracy'], label=f'{name} (Acc)')
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
plt.title('Accuracy vs Batch Size for Each Model')
plt.show()